In [1]:
from manim import *
import numpy as np
import pandas as pd
_30k = ["#202f66", "#ff7048", "#7f68d0", "#f3d36e", "#d869ab", "#48ADA9", "#1b262c"]

Manim Community v0.13.1

In [2]:
# manim video resolution config
config["frame_size"] = (2560, 1080)
config["frame_rate"] = 60
config["background_color"] = BLACK

In [3]:
%%manim -v WARNING MonteCarlo

config["frame_width"] = 21 * 1.25
config["frame_height"] = 9 * 1.25
# config["disable_caching_warning"] = True
config["save_last_frame"] = True
config["save_last_frame"] = False


class MonteCarlo(Scene):
    
    def construct(self):
        self.intro_scene()
        
        move_length = 6
        scale_size = 4
        
        # Monte Carlo Simulation Scene
        figure_group = circle, square =  self.draw_square_and_circle(scale_size=scale_size)
        figure_group.move_to(LEFT * move_length)
        
        # estimating pi formula
        formula = MathTex(r"\pi \approx 4 \cdot \frac{\text{\#dots\ in\ the\ circle}}{\text{\#total\ dots\ in\ the\ square}}")
        # formula = formula.to_edge(UP, buff=2)
        formula.next_to(RIGHT*move_length/3).shift(UP*2)
        
        
        
        ff = 'Gill Sans Nova'
        fs = 36
        estimated_pi = 0
        inner_dot = 0
        total_dot = 0
        n_dot = 15000
        
        inner_dots_group = inner_text, inner_count = VGroup(
            Text('#dots in the circle = ', font=ff, font_size=fs),
            DecimalNumber(inner_dot, num_decimal_places=0, font_size=fs)
        )
        inner_dots_group.arrange(RIGHT)
        inner_dots_group.next_to(formula, DOWN*2)
        
        total_dots_group = total_text, total_count = VGroup(
            Text('#total dots in the square = ', font=ff, font_size=fs),
            DecimalNumber(total_dot, num_decimal_places=0, font_size=fs)
        )
        total_dots_group.arrange(RIGHT)
        total_dots_group.next_to(inner_dots_group, DOWN)
        
        estimating_pi_group = pi_text, pi_val = VGroup(
            Text('Estimated pi = ', font=ff, font_size=fs),
            DecimalNumber(estimated_pi, num_decimal_places=4, font_size=fs)
        )
        estimating_pi_group.arrange(RIGHT)
        estimating_pi_group.next_to(total_dots_group, DOWN)
        
        text_group = VGroup(formula, inner_dots_group, total_dots_group, estimating_pi_group)
        
        
        self.play(Write(figure_group), Write(text_group))
        self.wait(2)
        
        # add updater
        pi_val.add_updater(lambda x: x.set_value(estimated_pi))
        total_count.add_updater(lambda x: x.set_value(total_dot))
        inner_count.add_updater(lambda x: x.set_value(inner_dot))
        
        
        # for loop here
        circle_center_x = circle.get_center()[0]
        circle_center_y = circle.get_center()[1]
        
        # slow motion for the first n dots
        for i in range(0, 100):
            xy = np.random.uniform(-1, 1, size=2)
            dot_color = self.square_color
            if np.sum(xy**2) <= 1:
                inner_dot += 1
                dot_color = self.circle_color
            
            total_dot += 1
            estimated_pi = 4*inner_dot/total_dot
            
            dot = Dot(point=[circle_center_x+xy[0]*scale_size, circle_center_y+xy[1]*scale_size, 0], color=dot_color, radius=0.04)
            
            # replace number
            self.play(Create(dot, run_time=0.05))
        
        # speed up the plot
        while total_dot < n_dot:
            dot_range = np.min([100, n_dot - total_dot])
            dot_group = []
            for i in range(0, dot_range):
                xy = np.random.uniform(-1, 1, size=2)
                dot_color = self.square_color
                if np.sum(xy**2) <= 1:
                    inner_dot += 1
                    dot_color = self.circle_color

                total_dot += 1
                estimated_pi = 4*inner_dot/total_dot

                dot = Dot(point=[circle_center_x+xy[0]*scale_size, circle_center_y+xy[1]*scale_size, 0], color=dot_color, radius=0.04)
                dot_group.append(dot)

                # replace number
            self.play(Create(VGroup(*dot_group), run_time=0.05))
        
        self.wait(2)
    
    def create_axes(self, show_number=True):
        
        step = 0.5
        axes_range = [-1.5, 1.5, step]
        axes_length = len(np.arange(*axes_range))
        one_unit_length = 1/step
        
        ax = Axes(
            x_range=axes_range,
            y_range=axes_range,
            axis_config={"include_numbers": show_number, "font_size": 24},
            x_length=axes_length,
            y_length=axes_length
        )
        
        return ax, one_unit_length
    
    def intro_scene(self):
        ax, one_unit_length = self.create_axes()
        
        self.circle_color = _30k[3]
        self.square_color = _30k[4]
        
        # scale to same size of an axes
        circle = Circle(radius=1, color=self.circle_color).scale(one_unit_length)
        square = SurroundingRectangle(circle, buff=0, color=self.square_color)
        
        # placeholder brace
        b = BraceBetweenPoints([0, 0, 0], [one_unit_length, 0, 0], direction=[0, -1, 0])
        brace = BraceLabel(b, 'r = 1', brace_direction=[0, 1, 0])
        
        circle_square_group = VGroup(circle, square)
        figure_group = VGroup(circle, square, brace, ax)
        
        self.play(Create(ax))
        self.play(Write(circle), Write(square))
        self.add(ax)
        self.play(Write(brace))
        self.wait(0.5)
        self.play(figure_group.animate.shift(LEFT * 5).scale(1.5))
        self.wait(1)
        
        circle_area_formula = circle_text, circle_area = VGroup(
            Text("Area of Circle = ", font_size=36, color=self.circle_color),
            MathTex("\pi r^2", color=self.circle_color)
        )
        circle_area.next_to(circle_text, RIGHT) 
        square_area_formula = square_text, square_area = VGroup(
            Text("Area of Square = ", font_size=36, color=self.square_color),
            MathTex("2r \cdot 2r = 4r^2", color=self.square_color)
        )
        square_area.next_to(square_text, RIGHT)
        square_area_formula.next_to(circle_area_formula, DOWN)
        
        formulae_group = VGroup(circle_area_formula, square_area_formula)
        formulae_group.move_to(RIGHT * 5)
        
        self.play(Write(formulae_group))
        self.play(formulae_group.animate.shift(UP * 2))
        self.wait()
        
        prob_group = prob_text, prob_formula = VGroup(
            Text("Prob(dots fall in the circle) = ", font_size=36),
            MathTex(r"\frac{\pi r^2}{4r^2}")
        )
        prob_formula.next_to(prob_text)
        prob_group.move_to(RIGHT * 5)
        self.play(Write(prob_group))
        self.wait()
        
        old_prob_formula = prob_formula
        prob_formula = MathTex(r"\frac{\pi}{4}").next_to(prob_text)
        self.play(ReplacementTransform(old_prob_formula, prob_formula))
        
        old_prob_group = VGroup(prob_text, prob_formula)
        prob_group = prob_text, prob_formula = VGroup(
            MathTex(r"\frac{\text{\#dots\ in\ circle}}{\text{\#total\ dots\ in\ square}} = "),
            MathTex(r"\frac{\pi}{4}")
        )
        prob_formula.next_to(prob_text)
        prob_group.move_to(RIGHT * 5)
        self.play(ReplacementTransform(old_prob_group, prob_group))
        
        condition = MathTex(r"\text{a dot (x, y) falls in the circle} \iff x^2 + y^2 \leqslant r^2", color=self.circle_color)
        condition.next_to(prob_group, DOWN)
        self.play(Write(condition))
        
        self.wait(1.5)
        self.play(FadeOut(prob_group, run_time=0.2), FadeOut(formulae_group, run_time=0.2), FadeOut(figure_group, run_time=0.2), FadeOut(condition, run_time=0.2))
    
    def draw_square_and_circle(self, scale_size=3):
        circle = Circle(radius=1, color=self.circle_color).scale(scale_size)
        square = SurroundingRectangle(circle, buff=0, color=self.square_color)
        
        return VGroup(circle, square)